# Analysis

In this section we will provide some descriptive statistics, including missing data, means and standard deviations, and a logistic regression on the full dataset.

In [1]:
using DataFrames
using CSV
using GLM
using Gadfly
using Statistics
using NamedArrays
#using DecisionTree

DATA="/Users/aguang/CORE/tippingpoint/tippingpoint/data"
df_dv = CSV.File(joinpath(DATA,"df_dv.dat"),normalizenames=true) |> DataFrame!
df_gc = CSV.File(joinpath(DATA,"df_gc.dat"),normalizenames=true) |> DataFrame!
df_subject = CSV.File(joinpath(DATA,"df_subject.dat"),normalizenames=true) |> DataFrame!
full_df = CSV.File(joinpath(DATA,"full_df.dat"),normalizenames=true) |> DataFrame!

,subj,graphid,tp,q,pt,risingBefore,cannotSeeAfter,downOverall
,Int64,String,Int64⍰,String,String,Int64,Int64,Int64
1,1,Q2 A,0,Q2,A,1,0,0
2,1,Q2 B,0,Q2,B,1,0,0
3,1,Q2 C,0,Q2,C,1,1,0
4,1,Q3 A,0,Q3,A,1,0,1
5,1,Q3 B,0,Q3,B,0,0,1
6,1,Q3 C,0,Q3,C,0,0,1
7,1,Q3 D,0,Q3,D,0,0,1
8,1,Q3 E,0,Q3,E,1,0,1
9,1,Q4 A,0,Q4,A,0,0,1


┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = compacttype(::Type, ::Int64) at show.jl:39
└ @ DataFrames /Users/aguang/.julia/packages/DataFrames/Iyo5L/src/abstractdataframe/show.jl:39


## Descriptive statistics

### Missing data

Here we filter out rows with missing values, as the function outputs missing otherwise. For the graph characteristics they should be coded as missing since reviewers were not able to reach an agreement. For the subject characteristics they are coded as missing because the individual did not answer the question.

There is a lot of missing data. 70 participants out of 178 did not respond to one or more of the questions in the survey. Most of the questions they didn't respond to were the slider scale questions about the importance of various graph characteristics. Not sure if there was an issue in filling that part of the survey out. We could choose to impute the missing values for the subject characteristics if desired, as they can be assumed to be missing at random. The "gold standard" for imputation is multiple imputation, where several different possible imputed data sets are created and results from each of them are combined.

Combined with 3 points of interest that had missing values associated with them (these were all in the complexity column) this became 2564 rows that had missing values in them, out of 5696.

In [ ]:
# how many rows with missing values in graph characteristics?
@show sum(.!completecases(df_gc))
# how many rows with missing values in subject characteristics?
@show sum(.!completecases(df_subject))
# how many rows with missing values in total data?
@show sum(.!completecases(full_df))
# what do the rows with missing values in subject characteristics look like?
df_subject[.!completecases(df_subject),8:15]

### Graph characteristic summaries

In total there were 32 points of interest across 13 graphs. 17 were coded as rising before point of interest, 4 as can't see graph after point of interest, 21 as overall trend of graph was going down, 2 as overall trend of graph was a bell curve, and 26 as graph was rated as complex.

In [ ]:
# how many types of graphs?
[sum(col) for col = eachcol(df_gc[completecases(df_gc),2:6])]

### Tipping point declarations

Number of tipping points declared was 2747, which was about half of the total number of observations.

**todo:** look more at points within graphs

In [ ]:
# how many tipping points declared?
@show sum(skipmissing(df_dv.tp))
# how many tipping points by point type?
@show plot(by(df_dv, :q, :tp => x -> sum(skipmissing(x))), x="q", y="tp_function", Geom.bar)
# how many tipping points by last point and previous points?
# have to do this code

### Correlation

There is also a modest degree of correlation among the independent variables. It would appear that both the graph rising before the point of interest and being unable to see the trend after the point of interest are negatively correlated to the overall graph going down. The correlation is likely due to there not being many points to evaluate. We can keep this in mind for interpreting results.

**todo:** check if correlation is statistically significant

In [ ]:
function corNamed(df, indices, names)
    cor = Statistics.cor(convert(Matrix, df[completecases(df),indices]))
    n = NamedArray(cor)
    setnames!(n, names, 1)
    return(n)
end

named_gc = corNamed(df_gc, 2:6, ["risingBefore", "cannotSeeAfter",
        "downOverall", "bellOverall", "complexOverall"])


Looking at correlation among variables relating to the subject, it appears that the university being Brown and the experience are highly correlated, which makes sense as the Executive Masters program is at Brown. Otherwise there does not appear to be much correlation.

In [ ]:
named_subj = corNamed(df_subject, 2:20, ["uniBrown", "expExec", "tpChange", "tpRate", "tpDir", "tpNoReturn",
            "tellMgr", "impChange", "impRise", "impFall", "impPeriodic", "numOtherTP",
            "liwcPosemo", "liwcNegemo", "liwcCause", "liwcFocusPre", "liwcFocusFut",
            "liwcRelativ", "liwcTime"])
@show named_subj[1.0 .> abs.(named_subj) .> 0.5]
named_subj

The full correlation matrix confirms that correlation is mostly seen between variables related to the graph characteristics, and between variables related to the subjects. Correlation among those groups appears to be minimal.

In [ ]:
named_cor = corNamed(full_df, 4:27, ["risingBefore", "cannotSeeAfter",
        "downOverall", "bellOverall", "complexOverall", "uniBrown", "expExec", "tpChange", "tpRate", "tpDir", "tpNoReturn",
            "tellMgr", "impChange", "impRise", "impFall", "impPeriodic", "numOtherTP",
            "liwcPosemo", "liwcNegemo", "liwcCause", "liwcFocusPre", "liwcFocusFut",
            "liwcRelativ", "liwcTime"])
#@show named_cor[1.0 .> abs.(named_cor) .> 0.5]
#named_cor

### Mean and standard deviation

Also, here is the mean and standard deviation similar to those in the 2 other papers. It does not look particularly informative to me.

In [ ]:
DataFrame(Variables=["risingBefore", "cannotSeeAfter",
        "downOverall", "bellOverall", "complexOverall", "uniBrown", "expExec", "tpChange", "tpRate", "tpDir", "tpNoReturn",
            "tellMgr", "impChange", "impRise", "impFall", "impPeriodic", "numOtherTP",
            "liwcPosemo", "liwcNegemo", "liwcCause", "liwcFocusPre", "liwcFocusFut",
            "liwcRelativ", "liwcTime"],
    Mean=[mean(skipmissing(col)) for col = eachcol(full_df[:,4:27])],
    SD=[std(skipmissing(col)) for col = eachcol(full_df[:,4:27])])

## Logistic regression

I have not looked at whether there are outliers in the data but given the correlation between the university and the amount of experience, I left the university variable out. Although there is correlation between the graph characteristics, again there are not that many points, so the correlation could be due to chance.

### Multiple logistic regression

Looking at the results and comparing them to the hypotheses, it looks like there may be evidence for **Hypothesis 1** (being unable to see what follows makes an individual less likely to declare a tipping point). The graph rising before the point in question is the strongest predictor of whether an individual will declare a tipping point or not, but it actually goes against **Hypothesis 2**.

In [ ]:
hp = glm(@formula(tp ~ risingBefore + cannotSeeAfter + downOverall + bellOverall +
        complexOverall + expExec + tpChange + tpRate + tpDir + tpNoReturn +
        tellMgr + impChange + impRise + impFall + impPeriodic +
        numOtherTP + liwcPosemo + liwcNegemo + liwcCause + liwcFocusPre + liwcFocusFut +
        liwcRelativ + liwcTime), full_df, Binomial(), LogitLink())

In [ ]:
hp = glm(@formula(tp ~ risingBefore + cannotSeeAfter + downOverall + bellOverall +
        complexOverall + expExec + tpChange + tpRate + tpDir + tpNoReturn +
        tellMgr + impChange + impRise + impFall + impPeriodic +
        numOtherTP + liwcPosemo + liwcNegemo + liwcCause + liwcFocusPre + liwcFocusFut +
        liwcRelativ + liwcTime), full_df[completecases(full_df),:], Binomial(), LogitLink())

In [ ]:
# Showing same information but as odds ratios which can be easier to interpret
# not declaring a tipping point is the reference
df_hp=DataFrame(OR=exp.(coef(hp)),Lower95=exp.(confint(hp)[:,1]),Upper95=exp.(confint(hp)[:,2]))
df_hp.Variable=["Intercept", "Rising Before", "Cannot see after", "Down overall",
    "Bell overall", "Complex overall", "Experience (Executive or Undergrad)",
    "tp change", "tp rate", "tpdir", "tp no return", "tell manager", "imp Change", "imp rise",
    "imp Fall", "impPeriodic", "numOtherTP", "liwcPosemo", "Negemo", "cause", "focus pre",
    "focus fut", "relative", "time"]
df_hp

When regressing on the variables individually, some of the variables are still significant, in particular those about the graph characteristics, but some of the variables are no longer significant. This suggests some kind of interaction effect. These results are not formatted super well, didn't have time to get to that.

In [ ]:
@show glm(@formula(tp ~ risingBefore), full_df, Binomial(), LogitLink())
@show glm(@formula(tp ~ cannotSeeAfter), full_df, Binomial(), LogitLink())
@show glm(@formula(tp ~ expExec), full_df, Binomial(), LogitLink())
@show glm(@formula(tp ~ complexOverall), full_df, Binomial(), LogitLink())
@show glm(@formula(tp ~ impChange), full_df, Binomial(), LogitLink())
@show glm(@formula(tp ~ tellMgr), full_df, Binomial(), LogitLink())
@show glm(@formula(tp ~ numOtherTP), full_df, Binomial(), LogitLink())

# Analyses for 9/24/19

## Dropping missing data

Since most of the missing data came from 9 variables: importance of sustained change, importance rise, importance fall, importance periodic nature of occurrence, LIWC def cause, LIWC def focuspresent, LIWC def focusfuture, LIWC def relativ, LIWC def time, we decided to drop them.

The results of the logistic regression after dropping the columns increases the significance of `expExec` which supports **Hypothesis 3:** more experienced subjects are less likely to declare a tipping point. It also increases the significance of `liwcPosemo`.

Interestingly, dropping those missing data columns led to the `tellMgr` random variable no longer being significant.

In [3]:
dropped = select(full_df, Not([:impChange,:impRise,:impFall,:impPeriodic,:liwcCause,:liwcFocusPre,:liwcFocusFut,:liwcRelativ,:liwcTime]))
hp_dropped = glm(@formula(tp ~ risingBefore + cannotSeeAfter + downOverall + bellOverall +
        complexOverall + expExec + tpChange + tpRate + tpDir + tpNoReturn +
        tellMgr + numOtherTP + liwcPosemo + liwcNegemo), dropped, Binomial(), LogitLink())

┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = (::getfield(StatsModels, Symbol("##40#41")){Array{Int64,1}})(::Array{Union{Missing, Int64},1}) at none:0
└ @ StatsModels ./none:0
┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = (::getfield(StatsModels, Symbol("##40#41")){Array{Int64,1}})(::Array{Int64,1}) at none:0
└ @ StatsModels ./none:0
┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = (::getfield(StatsModels, Symbol("##40#41")){Array{Int64,1}})(::Array{Float64,1}) at none:0
└ @ StatsModels ./none:0


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},LogitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

tp ~ 1 + risingBefore + cannotSeeAfter + downOverall + bellOverall + complexOverall + expExec + tpChange + tpRate + tpDir + tpNoReturn + tellMgr + numOtherTP + liwcPosemo + liwcNegemo

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                  Estimate  Std. Error    z value  Pr(>|z|)   Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)     -0.750149   0.237137    -3.16335     0.0016  -1.21493    -0.285369  
risingBefore     0.880678   0.0802935   10.9682      <1e-27   0.723306    1.03805   
cannotSeeAfter  -0.578849   0.117508    -4.92603     <1e-6   -0.809161   -0.348537  
downOverall      0.436222   0.115441     3.77873     0.0002   0.209961    0.66248

We run the analysis on each individual variable again using the filtered dataset. This time all the significant results that support **Hypothesis 1, 2, 3** stay significant.

In [ ]:
@show glm(@formula(tp ~ risingBefore), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ cannotSeeAfter), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ expExec), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ complexOverall), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ downOverall), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ tellMgr), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ numOtherTP), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ liwcPosemo), dropped, Binomial(), LogitLink())

# Interaction effects

Our next step is to look at potential interaction effects. The effects we are interested in are experience x emotions and curve rising x emotions. We do this both with logistic regression and decision trees.

## Decision Trees

In [5]:
using DecisionTree
dt = DecisionTreeRegressor()
test = convert(Matrix,dropped[:,[:risingBefore,:cannotSeeAfter,:downOverall,:expExec,:liwcPosemo,:liwcNegemo]])
tp = dropped[:,:tp]
DecisionTree.fit!(dt, test, tp)

LoadError: syntax: missing comma or ) in argument list

In [43]:
dt.left

ErrorException: type DecisionTreeRegressor has no field left

## Logistic regression

In [ ]:
@show glm(@formula(tp ~ expExec * liwcPosemo), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ expExec * liwcNegemo), dropped, Binomial(), LogitLink())

In [ ]:
@show glm(@formula(tp ~ risingBefore * liwcPosemo), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ risingBefore * liwcNegemo), dropped, Binomial(), LogitLink())

In [ ]:
@show glm(@formula(tp ~ cannotSeeAfter * liwcPosemo), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ downOverall * liwcPosemo), dropped, Binomial(), LogitLink())
@show glm(@formula(tp ~ bellOverall * liwcPosemo), dropped, Binomial(), LogitLink())
glm(@formula(tp ~ complexOverall * liwcPosemo), dropped, Binomial(), LogitLink())

# Next steps

 * ~~Drop slider data; will reduce amount of missing data~~ (Done 9/24/19)
 * ~~Keep in posemo negemo but drop the other liwc columns~~ (Done 9/24/19)
 * Look for interaction effects
 * Is the correlation statistically significant?
 * Any idea why rising before is so significant?
 * Thoughts on imputing missing data? And also the reason for missing data?
   - not going to impute
 * Thoughts on multicollinearity?
 * I still need to look at the other hypotheses, in particular 8 and 10.
 * Need to flesh out multiple logistic regression analysis, in particular likelihood ratio test and understanding any interaction effects.
 * One thing we might want to take a look at is if there is correlation among points on the same graph. For example, I would suspect that only one point on a single graph is actually marked as a tipping point.